## GPT-4o without Transkribus OCR


<img src="./data/01_quantitative/Seite0408.JPG" alt="Drawing" style="width: 200px;"/>

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI


load_dotenv()

client = OpenAI(api_key= os.getenv('OPENAI_API_KEY'))

In [6]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [8]:
from pydantic import BaseModel, Field

class TranscriptionPage(BaseModel):
    text: str = Field(..., description="The transcribed text of the page.")

In [9]:
base64_image = encode_image("./data/01_quantitative/Seite0408.JPG")

completion = client.beta.chat.completions.parse(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are an expert for handwritten Early Modern German. Transcribe the text in this image exactly and return it in markdown format."},
    {"role": "user", "content": [
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
            }
        ]}
  ],
  response_format=TranscriptionPage,
)

result = completion.choices[0].message.parsed

In [17]:
result.text
gpt4o_only = result.text
gpt4o_only

'ander Buch führen und\nbesser schreiben wird\nder Schul Capt:\nsuggerieret wurde, das ist\nanderer Schools Mann\naber hlicken ??? wachsen\nMögen bey dieser Zeit\nder anderen ???\nbelangen, da sie allein\nzu führen seyen, bei creación\ngeschriben \\ bey dem Bock\nwürden sich erforderen\nund berichtet von:\n?? überm Schwarzen'

## GPT-4o with Transkribus OCR - German Giant 1


In [13]:
transcribus_only = """
215
wider Ichtes firzenen
bedennckhlich werde
ai Ersamer Rath sich
och Zūr Zeit Zūgedūld
vnd erstbemelten Man¬¬
daten Zū beqūenen wissen.
Was beschliesslich
die. wider dactoon Ar¬
ara gefierte Clagan
belannge. seie selbiger
von Iren Gnaden dahi
beschailen. vnd beūelcht
worden. sich nit allain
deren Iir erlaūbten
wörn. vnd Pixen. mit
gebirenden discretion
gebraūchen, sonnder
aūch ii vbrigen ge¬
zimbtermassen. vnd"""

In [14]:
base64_image = encode_image("./data/01_quantitative/Seite0408.JPG")

completion = client.beta.chat.completions.parse(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": """You are an expert for handwritten Early Modern German. Transcribe the text in this image exactly and return it in markdown format.
     The following is the output of a Transkribus OCR model. It can be helpful with the transcription, but you should not rely on it completely as it can be wrong.
     {transcribus_only}
     """},
    {"role": "user", "content": [
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
            }
        ]}
  ],
  response_format=TranscriptionPage,
)

result = completion.choices[0].message.parsed

In [16]:
gpt4o_with_transkribus = result.text
gpt4o_with_transkribus

'ander Buch führen und\nbesser schreiben wird\nder Schul Capt:\nsuggerieret wurde, das ist\nanderer Schools Mann\naber hlicken ??? wachsen\nMögen bey dieser Zeit\nder anderen ???\nbelangen, da sie allein\nzu führen seyen, bei creación\ngeschriben \\ bey dem Bock\nwürden sich erforderen\nund berichtet von:\n?? überm Schwarzen'